In [1]:

from tqdm import tqdm
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from pathlib import Path
import spacy

In [ ]:
data_list = []

# TODO read it in as spacy docs, then remove digits, maybe remove verbs, then run the topic modeling, remove names

nlp = spacy.load("en_core_web_sm")


for path in Path("sloppy_ocr_manual_text").glob("*.txt"):
    with open(path, 'r', encoding='utf-8') as f:
        text = f.read()
        text = text.replace("\n", " ") #just in case some nonsense occurred in the ocr
    
    data_list.append((path.name, text))

In [3]:
df = pd.DataFrame(data_list, columns=['file_name', 'text'])

In [4]:
vectorizer = TfidfVectorizer(min_df=0.1, max_df=0.8, stop_words="english")
vectorized_data = vectorizer.fit_transform(df["text"])

In [5]:
nmf = NMF(n_components=10, random_state=42)

In [14]:
doc_topic_distribution = nmf.fit_transform(vectorized_data)

In [15]:
topic_words_df = pd.DataFrame(nmf.components_ , columns=vectorizer.get_feature_names_out())

In [16]:
topics = []
for topic, topic_row in topic_words_df.iterrows():
    top_10_words = topic_row.sort_values(ascending=False).head(10).index.tolist()
    topic_label = ", ".join(top_10_words)
    print(f"Topic {topic}: {topic_label}")
    topics.append(topic_label)


Topic 0: kim, minjung, seoul, yi, contemporary, lee, western, museum, exhibition, paintings
Topic 1: japanese, imperial, difference, modernism, dissertation, local, color, picturesque, east, landscape
Topic 2: dansaekhwa, movement, performative, endurance, kukje, labor, abstraction, essay, material, action
Topic 3: material, surface, artifacts, monochrome, remains, koreanness, historical, tansaekhwa, paper, surfaces
Topic 4: film, cinema, films, hong, soo, lee, sang, dong, box, poetry
Topic 5: contemporary, horlyck, war, kee, ideology, north, chapters, book, argues, suspended
Topic 6: self, kisaeng, portraiture, female, beauty, chapter, figure, women, portrait, western
Topic 7: research, laboratory, table, woman, yi, nihonga, late, glass, ae, experiment
Topic 8: experimental, kim, class, work, middle, state, university, society, does, mean
Topic 9: ink, jung, abstraction, like, suh, euro, contemporary, nature, work, bienal


In [24]:
#TODO 
doc_topic_df = pd.DataFrame(doc_topic_distribution, columns=topics)

for doc_index, doc_row in doc_topic_df.iterrows():
    top_3_topics = doc_row.sort_values(ascending=False).head(3)

    doc_info = df.loc[doc_index, ['file_name']]
    print(f"\n Top 3 topics for file {doc_info['file_name']}: ")
    for topic, weight in top_3_topics.items():
        print(f'{topic} {weight}')


 Top 3 topics for file art_of_dansaekhwa_press_release.txt: 
dansaekhwa, movement, performative, endurance, kukje, labor, abstraction, essay, material, action 0.7960762341774905
kim, minjung, seoul, yi, contemporary, lee, western, museum, exhibition, paintings 0.02861283126457418
self, kisaeng, portraiture, female, beauty, chapter, figure, women, portrait, western 0.0010412138323910938

 Top 3 topics for file Book_Reviews—Korea_Korean_Art.txt: 
contemporary, horlyck, war, kee, ideology, north, chapters, book, argues, suspended 0.600944477849348
dansaekhwa, movement, performative, endurance, kukje, labor, abstraction, essay, material, action 0.08469338325537273
japanese, imperial, difference, modernism, dissertation, local, color, picturesque, east, landscape 0.028785124472498095

 Top 3 topics for file companion_to_korean_art_chap15.txt: 
kim, minjung, seoul, yi, contemporary, lee, western, museum, exhibition, paintings 0.38373858770362196
self, kisaeng, portraiture, female, beauty, c

In [20]:
df

,file_name,text
0,art_of_dansaekhwa_press_release.txt,OCULA GROUP EXHIBITION The Art of Dansaekhwa ...
1,Book_Reviews—Korea_Korean_Art.txt,Korean Art: From the 19th Century to the Prese...
2,companion_to_korean_art_chap15.txt,15Modern Korean Art in the Japanese Colonial P...
3,companion_to_korean_art_chap16.txt,16The Long Breath: Postwar Korean Art Joan Kee...
4,companion_to_korean_art_chap17.txt,"17Situating Contemporary Art of South Korea, 1..."
5,companion_to_korean_art_chap18.txt,18South Korean Cinema in the Age of Hallyu Kyu...
6,contemporary_art_korea.txt,What Contemporary Art Means in Korea Joan Kee...
7,early_20th_century_japanese_and_korean_art.txt,Notes to Reader All Japanese and Korean names...
8,early_20th_century_japanese_and_korean_art_cha...,CHAPTER ONE In Search of the Picturesque East ...
9,early_20th_century_japanese_and_korean_art_cha...,CHAPTER TWO Self-Portraiture and the Figure of...
